# 인프라를 활용한 HTTPS 연결 구현

## HTTPS란?
- HTTP의 전송 중 데이터 유출을 방지하기 위해 통신에 보안을 추가한 프로토콜

## 일반적인 HTTPS 제공 방법
- 일반적인 순서
    - HTTPS 인증서 구매/발급(10불정도)
    - 인증서 병합 및 설정
    - 웹서버에 인증서 설치
    - 웹서버 설정 변경
        - HTTPS 적용
        - HTTPS 리다이렉션 등

## AWS 인프라를 활용한 HTTPS 구현
- AWS 인프라(CloudFront, ALB)를 활용하여 HTTPS 연결을 제공하는 방식
    - 둘 다 Route53 혹은 다른 방법의 도메인 제공 및 인증 필요
- 장점
    - 인증서 비용 무료
    - 여러 웹사이트에 적용 가능하며 대상/Origin 변경 등에 영향을 안받음
    - 인증서 자동 갱신 등 관리 편의성
- 기타
    - 조금의 추가 비용 발생
    - 경우에 따라 HTTPS 통신이 퍼블릭에 노출될 수 있음

## Amazon CloudFront를 활용한 HTTPS 구현
- 기본적으로 배포 생성 시 클라우드프론트의 아이디가 포함된 도메인 제공
    - 예 : d1234avcd.cloudfront.net
- 별도로 보유한 도메인을 부여해 클라우드프론트와 연결 가능
    - Route53 이나 다른 Registrar을 통하여 도메인 보유 필요
    - 이 때, 선택적으로 HTTPS를 사용하여 클라우드프론트에 접근하도록 설정 가능
        - HTTP/HTTPS 사용, HTTP를 HTTPS로 Redirect, HTTPS만 사용하는 세가지 모드가 있음
- 사용 사례
    - S3 Static Hosting 웹사이트의 HTTPS 제공
    - 단순한 웹사이트 등의 HTTPS 혹은 ALB등의 Origin을 둘 때 HTTPS를 CloudFront에서 받고 싶은 경우

## CloudFront의 HTTPS 프로토콜 활용
- ACM에서 SSL 인증서 발급 or Import 필요(US-East-1)
- HTTPS 가 HTTPS에 비해 조금 더 비쌈
- 두 가지 모드
    - SNI 지원 클라이언트만 가능 : 무료, 단 예전 브라우저의 경우 지원하지 않을 수 있음
    - ALL Client : 유료(600달러/월), 모든 클라이언트를 지원하지만 클라우드프론트에 전용 아이피 주소 부여 필요

## AWS Certificate Manager
- AWS Certificate Manager는 AWS 서비스 및 연결된 내부 리소스에 사용할 공인 및 사설 SSl/TLS(Secure Socket Layer/전송 계층 보안) 인증서를 손쉽게 프로비저닝, 관리 및 배포할 수 있도록 지원하는 서비스입니다
- AWS에서 SSL/HTTPS에 사용하는 인증서를 관리하는 서비스
    - 인증서를 발급받거나 Import 가능
- ALB, CloudFront, API Gateway와 연동하여 쉽게 HTTPS 프로토콜 구현 가능
- 인증서 Export 불가능
    - 즉 ALB, CloudFront, API Gateway, AWS Cognito 등 AWS 서비스만 사용 가능
- 무료

## Application Load Balacer를 활용한 HTTPS 제공
- ALB의 리스너에서 443 포트를 받아 처리
    - HTTPS 통신을 ALB에서 받아 HTTP 프로토콜로 타겟 그룹에 전달하는 방식
- ACM으로 SSL 인증서 Import 필요 (ALB가 있는 리전)
    - 혹은 원한다면 자신이 보유한 인증서 Import 가능
- 주의할 점
    - 인프라에서 Target/Origin 까지 HTTP로 연결
        - ALB 는 Private 통신이 가능하기에 큰 문제는 없으나, CloudFront의 경우 Public 인터넷 통과
    - 실제 Origin/Target 입장에서는 HTTP로 받기 때문에 HTTPS로 리다이렉션 등의 설정이 있는 경우 무한 리다이렉트 등의 문제 발생

### 실습 1 - CloudFront를 활용한 HTTPS 구현
1. 인스턴스 프로비전
    - 키페어 없음
    - 보안그룹 디폴트
    - 유저데이터에 스크립트 입력
2. 버킷 프로비전
    - 버킷에 index.html 업로드
3. ACM으로 인증서 만들기
    - Certificate Manager : 버지니아 북부 리전 선택
    - 퍼블릭 인증서 요청
    - apex 도메인(lumifla.com), *.lumifla.com 도 입력
    - Route53에서 레코드 생성
4. 클라우드프론트로 이동, ec2 오리진 배포 생성
    - 오리진 : 인스턴스
    - 프로토콜 : http
    - 뷰어 프로토콜 : Redirect
    - 캐시 옵션 레거시, 커스터마이즈, TTL 0초
    - 보안 보호 비활성화
    - 대체 도메인 이름 : ec2.lumifla.com
    - 인증서 선택
5. S3 오리진 배포 생성
    - 오리진 : S3 버킷 이름
    - 원본 엑세스 제어 설정
        - Create New OAC
    - Redirect HTTP to HTTPS
    - 캐시 옵션 레거시, 커스터마이즈, TTL 0초
    - 보안 보호 비활성화
    - 대체 도메인 이름 : s3.lumifla.com
    - 인증서 선택
6. 버킷 정책 업데이트
7. Route53 레코드 생성(s3)
    - 레코드 이름 : s3
    - 별칭
    - 엔드포인트 : 클라우드프론트
8. Route53 레코드 생성(ec2)
    - 레코드 이름 : ec2
    - 별칭
    - 엔드포인트 : 클라우드프론트
9. s3.lumifla.com/index.html로 접속해보자
    - 안전한 연결 확인(인증서 유효)
10. ec2.lumifla.com도 HTTPS 연결이 되는지 확인해보자

    


### 실습 2 - ALB를 이용한 HTTPS 구현
1. EC2 - 대상 그룹 생성
    - 상태검사 : /index.html
2. 로드밸런서 생성
    - ALB 선택
    - 가용 영역 전부 선택
    - 보안그룹 디폴트
    - 리스너 http만 선택 + Target Group 선택
3. 인증서 생성 (서울 리전)
4. ALB에 리스너 추가
    - 프로토콜 https
    - 대상 그룹 동일
    - 보안 리스너에서 인증서 선택
5. Route53 레코드 생성
    - alb
    - 별칭
    - ALB : 서울 리전
    - alb.lumifla.com 으로 접속해보자 - HTTPS 확인

### 리소스 정리
- 로드 밸런서, 인스턴스, 클라우드프론트 배포, 버킷